### Como da muchos errores voy a realizar la ejecición por comando del algortitmo

In [29]:
import aide
import os
import openai

Las celdas que voy a ajecutar a continuación precisan de tener las librerías instaladas así como la clave de de OpenAI correctamente exportada

In [32]:
client = openai.OpenAI()
modelos = client.models.list().to_dict()
modelos = modelos['data']

In [33]:
for modelo in modelos:
    print(modelo['id'])

dall-e-3
gpt-4-1106-preview
whisper-1
davinci-002
babbage-002
dall-e-2
gpt-3.5-turbo-16k
tts-1-hd-1106
tts-1-hd
gpt-3.5-turbo-1106
gpt-4o-2024-05-13
gpt-4o
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-instruct
text-embedding-3-small
gpt-4-turbo-2024-04-09
tts-1
gpt-4-turbo
gpt-4-0125-preview
gpt-3.5-turbo-0125
gpt-3.5-turbo
gpt-3.5-turbo-0301
gpt-4-turbo-preview
tts-1-1106
text-embedding-3-large
gpt-3.5-turbo-0613
gpt-4
text-embedding-ada-002
gpt-4-1106-vision-preview
gpt-3.5-turbo-16k-0613
gpt-4-0613
gpt-4-vision-preview


Estos son todos los modelos que podemos usar

Para cambiar correctamente el LLE que queremos usar en nuestro AIDE solo tenemos que dirigirnos al fichero alojado en nuestro 'environment' donde tenemos alojada la librería, lo explico en mi caso y ya pueden moverse donde necesiten.

En mi caso es ".aide-venv/lib/python3.10/site-packages/aide/utils/config.yaml" teniéndolo en un 'environment' aislado para no tener problemas con otras librerías, dentro de este fichero de configuración tenemos que ir a las líneas de código que pone model: gpt-4-turbo y cambiarlo por cualquiera de los que tengamos arriba en la lista de modelos

In [34]:
test = aide.Experiment(
    data_dir="Data/",
    goal="Quiero que me hagas un clasificador que sea capaz de predecir correctamente los tweets en español del fichero sem_eval_train_es.csv que tienes usando el modelo de lenguaje es_core_news_test de spacy que ya está instalado",
    eval="F1"
)

In [35]:
mejor_solucion = test.run(steps=10)

/Users/alejandro/Documents/Repositorios/alejandromadrid/.aide-venv/lib/python3.10/site-packages/aide/utils/tree_export.py:34: RuntimeWarning: invalid value encountered in divide
  layout = (layout - layout.min(axis=0)) / (layout.max(axis=0) - layout.min(axis=0))


In [36]:
print("La mejor solución tiene como matriz de validación: {}".format(mejor_solucion.valid_metric))
print("La mejor solución tiene por código: {}".format(mejor_solucion.code))

La mejor solución tiene como matriz de validación: 0.34067085953878407
La mejor solución tiene por código: import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import f1_score
from sklearn.multioutput import MultiOutputClassifier

# Load data
train_data = pd.read_csv("./input/sem_eval_train_es.csv")
test_data = pd.read_csv("./input/sem_eval_test_grupo_10.csv")

# Preprocess tweets
train_data["Tweet"] = train_data["Tweet"].str.lower()
test_data["Tweet"] = test_data["Tweet"].str.lower()

# Define features and labels
X = train_data["Tweet"]
y = train_data.iloc[:, 2:]

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(X)

# Define the model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_rf = MultiOutputClassifier(rf, n_jobs=-1)

# Evaluate with 5-fold cross-

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import f1_score
from sklearn.multioutput import MultiOutputClassifier
import pandas as pd

# Load data
train_data = pd.read_csv("./Data/sem_eval_train_es.csv")
test_data = pd.read_csv("./Data/sem_eval_test_grupo_10.csv")

# Preprocess tweets
train_data["Tweet"] = train_data["Tweet"].str.lower()
test_data["Tweet"] = test_data["Tweet"].str.lower()

# Define features and labels
X = train_data["Tweet"]
y = train_data.iloc[:, 2:]

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(X)

# Define the model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_rf = MultiOutputClassifier(rf, n_jobs=-1)

# Evaluate with 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
y_pred = cross_val_predict(multi_target_rf, X_tfidf, y, cv=kf)

# Compute F1 score
f1 = f1_score(y, y_pred, average="micro")
print(f"Validation F1 Score: {f1}")

# Train on full data
multi_target_rf.fit(X_tfidf, y)

# Predict on test data
X_test_tfidf = tfidf.transform(test_data["Tweet"])
test_preds = multi_target_rf.predict(X_test_tfidf)

# Save test predictions
submission = pd.DataFrame(test_preds, columns=y.columns)
submission.insert(0, "ID", test_data["ID"])
submission.to_csv("./Data/submission.csv", index=False)


Validation F1 Score: 0.34067085953878407


OSError: Cannot save file into a non-existent directory: 'working'

#### El coste de la consulta usando gpt-4o ha sido de $0.87